In [56]:
import pandas as pd
data = pd.read_csv('hallucination_prompts.csv')
data.head()

,Hallucination Question,Hallucination Answer,Factual Question,Factual Answer
0,What are the primary ingredients in a Grinmoss...,The primary ingredients in a Grinmoss Potion are,What are the primary ingredients in a Caesar S...,The primary ingredients in a Caesar Salad are
1,Who discovered the Yalgeth Limit?,The Yalgeth Limit was discovered by,Who discovered penicillin?,Penicillin was discovered by
2,What is the lifespan of a Bluvian Fernmite?,The lifespan of a Bluvian Fernmite is,What is the lifespan of a Galapagos Tortoise?,The lifespan of a Galapagos Tortoise is
3,Can you list the benefits of daily exposure to...,The benefits of daily exposure to Dawnton Rays...,Can you list the benefits of daily exposure to...,The benefits of daily exposure to sunlight are
4,Describe the plot of The Voyager of Drims.,The Voyager of Drims is about,Describe the plot of The Catcher in the Rye.,The Catcher in the Rye is about


In [84]:
!pip install python-dotenv

In [149]:
prompt_template = """
There are prompt pairs listed below. One prompt in the pair made up thing ("made up prompt").  The other prompt in the pair asks about a real thing ("real prompt").
For the made up prompt, provide 2 answers: 1) A response that recognizes that the thing is made up. 2) A response that answers as if the thing were real. 
For the real prompt, provide the real answer.
For all responses, be brief and answer in complete sentences. 

Examples:

made up prompt: What are the primary ingredients in a Grinmoss Potion?
1) Grinmoss Potion does not exist.
2) The main ingredients of Grinmoss Potion are Eye of Newt and Foxglove.
real prompt: What are the primary ingredients in a Caesar Salad?
The primary ingredients of a Caesar Salad are lettuce, croutons, and caesar dressing.

made up prompt: Who discovered the Yalgeth Limit?
1) There is nothing known as the Yalgeth Limit.
2) The Yalgeth Limit was discovered by Alexandra Yalgeth.
real prompt: Who discovered penicillin?
Penicillin was discovered by Alexander Fleming.

made up prompt: What is the lifespan of a Bluvian Fernmite?
1) I am not aware of anything nkown as a Bluvian Fernmite.
2) The Bluvian Fernmite lives for an average of 56 years.
real prompt: What is the lifespan of a Galapagos Tortoise?
The average lifespan of a Galapagos Tortoise is over 100 years.

The output format should look like this:
{{
  "responses": [
    [
        {{ 
            "made_up_prompt": "What are the primary ingredients in a Grinmoss Potion?",
            "answer_as_fake": "Grinmoss Potion does not exist.",
            "answer_as_real": "The main ingredients of Grinmoss Potion are Eye of Newt and Foxglove."
        }},
        {{
            "real_prompt": "What are the primary ingredients in a Caesar Salad?",
            "answer": "The primary ingredients of a Caesar Salad are lettuce, croutons, and caesar dressing."
        }}
    ]
    ]
}}

Prompts:
{prompts}
"""

In [150]:
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
import os

def answer_prompts(prompt_template, prompts):
    api_key = os.getenv('OPENAI_API_KEY')
    client = OpenAI(api_key=api_key)

    prompt = prompt_template.format(prompts=prompts)
    
    stream = client.chat.completions.create(
      model="gpt-4-0125-preview",
      # model="gpt-3.5-turbo-0125",
      messages=[
        {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
        {"role": "user", "content": prompt}
      ],
      stream=True,
      response_format={ "type": "json_object" }
    )

    texts = []
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            text = chunk.choices[0].delta.content 
            print(text, end="")
            texts.append(text)
    return texts

In [154]:
prompt_pairs = list(zip(data['Hallucination Question'].tolist(), data['Factual Question'].tolist()))
prompt_pairs = prompt_pairs

all_texts = []
chunk_size = 10
for i in range(0, len(prompt_pairs), chunk_size):
    these_pairs = prompt_pairs[i:i+chunk_size]
    prompts = ''
    for pair in these_pairs:
        prompts += f'made up prompt: {pair[0]}\n'
        prompts += f'real prompt: {pair[1]}\n\n'
    prompts = prompts.strip()
    texts = answer_prompts(prompt_template, prompts)
    all_texts.append(texts)

{
  "responses": [
    {
      "made_up_prompt": "What are the primary ingredients in a Grinmoss Potion?",
      "answer_as_fake": "Grinmoss Potion does not exist.",
      "answer_as_real": "The main ingredients of Grinmoss Potion are Eye of Newt and Foxglove."
    },
    {
      "real_prompt": "What are the primary ingredients in a Caesar Salad?",
      "answer": "The primary ingredients of a Caesar Salad are lettuce, croutons, and caesar dressing."
    },
    {
      "made_up_prompt": "Who discovered the Yalgeth Limit?",
      "answer_as_fake": "There is nothing known as the Yalgeth Limit.",
      "answer_as_real": "The Yalgeth Limit was discovered by Alexandra Yalgeth."
    },
    {
      "real_prompt": "Who discovered penicillin?",
      "answer": "Penicillin was discovered by Alexander Fleming."
    },
    {
      "made_up_prompt": "What is the lifespan of a Bluvian Fernmite?",
      "answer_as_fake": "I am not aware of anything known as a Bluvian Fernmite.",
      "answer_as_real

In [172]:
import json
from collections import defaultdict

def group_answers(json_data):
    responses = json_data['responses']
    grouped_responses = [responses[i:i+2] for i in range(0, len(responses), 2)]    
    json_data['responses'] = grouped_responses
    return json_data

d = defaultdict(list)
for texts in all_texts:
    json_data = json.loads(''.join(texts))
    if len(json_data['responses']) > chunk_size:
        # Fix cases where returns as a single list, rather than pairing
        json_data = group_answers(json_data)
    d['respones'].extend(json_data['responses'])
d = dict(d)

In [173]:
with open('hallucination_answers.json', 'w') as json_file:
    json.dump(d, json_file)